In [ ]:
!pip install konlpy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import os
import re
from sklearn.model_selection import train_test_split
from konlpy.tag import Okt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 17.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [ ]:
# 태그 단어
PAD = "<PADDING>"   # 패딩
STA = "<START>"     # 시작
END = "<END>"       # 끝
OOV = "<OOV>"       # 없는 단어(Out of Vocabulary)

# 태그 인덱스
PAD_INDEX = 0
STA_INDEX = 1
END_INDEX = 2
OOV_INDEX = 3

# 데이터 타입
ENCODER_INPUT  = 0
DECODER_INPUT  = 1
DECODER_TARGET = 2

# 한 문장에서 단어 시퀀스의 최대 개수
max_sequences = 30

# 임베딩 벡터 차원
embedding_dim = 100

# LSTM 히든레이어 차원
lstm_hidden_dim = 128

# 정규 표현식 필터
RE_FILTER = re.compile("[.,!?\"':;~()]")

# Data Frame Create
story_df = pd.read_csv('/content/drive/MyDrive/my_ws/project/Final/Data/story.csv', encoding='cp949')

# 1. 정규식을 사용해서 특수문자, 숫자 제거
# story_df의 각 열에 대해서 정규 표현식을 사용하여 문자열 치환을 수행하는 반복문입니다.
for i in range(10):  # 0부터 9까지의 열 인덱스
    story_df[str(i)] = story_df[str(i)].str.replace('[^A-Za-z가-힣ㄱ-ㅎㅏ-ㅣ ]', '')

story_df.drop(['결론'], axis=1, inplace=True)

# 결과를 저장할 빈 데이터프레임 리스트를 초기화합니다.
dataframes = []

# story_df에서 9열까지 있다고 가정합니다 (0부터 9까지, 총 10개의 열).
# 0열부터 8열까지를 input으로 하고, 그 다음 열을 output으로 하는 데이터프레임을 생성합니다.
for i in range(9):  # 0부터 8까지 (9는 포함되지 않음)
    # 각 행에 대해 원하는 열을 공백으로 구분하여 합치기 위한 람다 함수를 apply 메서드에 사용합니다.
    input_column = story_df.iloc[:, :(i + 1)].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
    output_column = story_df.iloc[:, i + 1]

    # 새로운 데이터프레임을 만들고 dataframes 리스트에 추가합니다.
    df = pd.DataFrame({'input': input_column, 'output': output_column})
    dataframes.append(df)

# 모든 데이터프레임을 세로 방향으로 합쳐서 result_df를 생성합니다.
result_df = pd.concat(dataframes, ignore_index=True)

# 결과를 출력합니다.
result_df

# x_data_df = result_df['input'];
# t_data_df = result_df['output'];

# x_data_train, x_data_test, t_data_train, t_data_test = \
# train_test_split(x_data_df,
#                  t_data_df,
#                  test_size=0.2,
#                  random_state=42)

# question, answer = list(x_data_train), list(t_data_train)
# # 챗봇 데이터 출력
# for i in range(5):
#     print('Q : ' + question[i])
#     print('A : ' + answer[i])
#     print()

<ipython-input-2-66dc1e2ed1af>:36: FutureWarning: The default value of regex will change from True to False in a future version.
  story_df[str(i)] = story_df[str(i)].str.replace('[^A-Za-z가-힣ㄱ-ㅎㅏ-ㅣ ]', '')


,input,output
0,메모리 오브 더 스타는 과학자인 주인공 조나단의 이야기를 그린 영화입니다,조나단은 별들이 가진 기억을 추출할 수 있는 기계를 개발하게 됩니다
1,세상에서 가장 큰 도시 뉴욕에서 살던 주인공 존은 일상에 지쳐 있었습니다,그는 어느 날 친구에게서 고요한 시골 마을에 있는 오래된 집을 물려받게 되었습니다
2,스카이라는 이름의 소녀는 평범한 가정에서 자라났지만 늘 독특한 능력을 가지고 있었습니다,그녀는 생각을 읽는 능력을 가지고 있었는데 이는 그녀가 태어날 때부터 가지고 있던 ...
3,영화의 주인공인 제이크는 평범한 일상을 보내던 중 고요한 작은 마을에서 발견된 미스...,그 곳은 현실과는 다른 황홀한 세계로 시간과 공간이 완전히 달랐습니다
4,빛의 도시는 찬란한 미래 도시를 배경으로 한 영화입니다,주인공은 미래 과학을 이용해 시간 여행을 하는 천재 과학자 제이크입니다
...,...,...
895,마을의 평범한 청년 민호는 어느 날 우연히 숲에서 빛나는 돌을 발견한다 그 돌은 예...,그는 모든 동료들에게 감사의 마음을 전하며 마을로 돌아가게 된다
896,대도시에서 살아가던 한 소녀 수아는 어느 날 의문의 편지를 받게 된다 그 편지는 수...,그녀는 도시에서의 삶에 새로운 의미를 찾게 되고 그녀의 삶은 더욱 풍요로워진다
897,이야기는 작은 도시 속 평범한 가게 주인인 제임스의 일상에서 시작한다 그러던 어느 ...,그들은 유물을 안전하게 되돌려 놓고 평범한 일상으로 돌아온다
898,영화는 바다를 사랑하는 소녀 미나의 꿈을 그리며 시작된다 미나는 어릴 적부터 세계를...,세계에서 가장 아름다운 바다가 복원된 후 미나와 물고기는 각자의 길을 간다


In [ ]:
import pandas as pd
import torch
from transformers import T5TokenizerFast, T5ForConditionalGeneration, AdamW
from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset

# 디바이스 설정
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 데이터셋 클래스 정의
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        questions = self.data.iloc[idx]['input']
        answers = self.data.iloc[idx]['output']

        # 토큰 수를 max_length에 맞춰서 자르거나 패딩
        inputs = self.tokenizer(questions, max_length=self.max_length, truncation=True, padding='max_length', return_tensors='pt')
        labels = self.tokenizer(answers, max_length=self.max_length, truncation=True, padding='max_length', return_tensors='pt')

        return {
            'input_ids': inputs.input_ids[0],
            'labels': labels.input_ids[0]
        }

# 데이터 로드 및 전처리
df = result_df

# 토크나이저와 모델 로드
model_name = '/content/drive/MyDrive/my_ws/project/Final/pko-t5-base'  # 구글드라이브
# model_name = "paust/pko-t5-base"  # 직접다운로드

tokenizer = T5TokenizerFast.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
model.to(device)



In [ ]:
# 학습 설정
optimizer = AdamW(model.parameters(), lr=1e-4)
criterion = torch.nn.CrossEntropyLoss()

num_epochs = 1
log_interval = 100
checkpoint_interval = 0.5  # 0.5 에폭마다 체크포인트 저장

# DataLoader를 사용하여 데이터 로드
batch_size = 4
dataset = CustomDataset(df, tokenizer)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# 학습 루프에서 데이터 로더 사용
total_steps = 0
for epoch in range(num_epochs):
    for batch in tqdm(data_loader, total=len(data_loader)):
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)

        # 모델 학습
        outputs = model(input_ids=input_ids, labels=labels)
        loss = outputs.loss

        # 역전파 및 가중치 업데이트
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_steps += 1

        # 0.5 에폭마다 체크포인트 저장
        if total_steps % int(len(dataset) / (batch_size * 2)) == 0:
            checkpoint_dir = f"checkpoint_{total_steps}"
            model.save_pretrained(checkpoint_dir)

# 학습이 완료된 모델 저장
model.save_pretrained("2save")

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
100%|██████████| 225/225 [01:23<00:00,  2.70it/s]


In [ ]:

user_input = "군 복무를 마치고 복학한 만섭은 학교에 족구장이 사라졌다는 걸 알게 된다. 만섭은 처음에는 이를 믿지 않지만 함께 시간을 보내기로 결심한다. 만섭은 그 사실을 안 순간부터 그녀를 의심하기 시작한다. 처음에는 그녀의 말에 솔깃해진 나머지 그와 함께 족구장을 찾아 나선다. 하지만 그들은 마지막 순간 위험을 함께 극복하며 새로운 희망을 찾는다. 하지만 그들은 마지막 순간까지 서로를 믿지 않는다"

prompt = f"{user_input}"
input_ids = tokenizer(prompt, return_tensors='pt').input_ids.to(device)

logits = model.generate(
        input_ids,
        max_length=1024,
        temperature=0.5,
        no_repeat_ngram_size=6,
        do_sample=True,
        num_return_sequences=1,
    )
text = tokenizer.batch_decode(logits, skip_special_tokens=True)[0]

print(text)

결국 그들은 족구장에서 새로운 희망을 찾기로 결정한다
